In [ ]:
from ibm_watson import SpeechToTextV1                           # Read a speech file and take a json of it.
from ibm_watson import LanguageTranslatorV3                     # Translate a text and take a json with the translation.
from ibm_watson import TextToSpeechV1                           # Parse text to Watson and take a sound file.
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator  # IBM Authenti object
import Keys                                                     # API keys.
import pyaudio                                                  # Use of our mic to record audio.
import pydub                                                    # Load a .wav file.
import pydub.playback                                           # play a .wav file.
import wave                                                     # save a .wav file

In [ ]:
def RunTranslator() :
    """
     Driver Program.
    """
    # A message to the user.
    input("Press enter and start to talk -> Ask your question.")
    
    # Use the RecordAudio function to create a .wav file.
    RecordAudio("English.wav") 
    
    # Take the english text from the .wav file, using the model "en-US_BroadbandModel".
    FileInEnglish = SpeechToText("English.wav", "en-US_BroadbandModel")
    print("English: ", FileInEnglish)
    
    # Take the english text and translate it to Spanish.
    FileInSpanish = Translate(FileInEnglish, "en-es")
    print("Spanish: ", FileInSpanish)
    
    # Take the text and make it into a .wav with Sofia's voice.
    TextToSpeech(FileInSpanish, "es-US_SofiaVoice", "Spanish.wav")
    
    # Play the spanish translated audio.
    PlayAudio("Spanish.wav")
    
    # A message to the user.
    input("Press enter and start to talk -> Make a response in spanish.")
    
    # Use the RecordAudio function to create a .wav file with spanish answer.
    RecordAudio("SpanishAnswer.wav") 
    
    # Take the spanish text from the .wav file, using the model "en-ES_BroadbandModel".
    FileInSpanish = SpeechToText("SpanishAnswer.wav", "es-ES_BroadbandModel")
    print("SpanishResponse: ", FileInSpanish)
    
    # Take the spanish text and translate it to english.
    FileInEnglish = Translate(FileInSpanish, "es-en")
    print("EnglishResponse: ", FileInEnglish)
    
    # Take the text and make it into a .wav with Allison's voice.
    TextToSpeech(FileInEnglish, "en-US_AllisonVoice", "EnglishAnswer.wav")
    
    # Play the response in english.
    PlayAudio("EnglishAnswer.wav")

In [ ]:
def SpeechToText(FileName, Model) :
    """
     A function which converts Speech to Text(.json).
    """
    
    # Create a SpeechToText object using our API key.
    Authenticator = IAMAuthenticator(Keys.IBM_SpeechToTextAPIKey)
    STT = SpeechToTextV1( authenticator = Authenticator)
    STT.set_service_url(Keys.IBM_SpeechToTextURL)
    
    # Connect to our audio file and get the .json to Result.
    with open(FileName, "rb") as AudioFile :
        Result = STT.recognize(audio = AudioFile, 
                               content_type = "audio/wav", model = Model).get_result()
        
    # Get the results from the .json.
    ResultList = Result["results"]
    
    # The final Speech Recognition Result.
    SpeechRecognitionResult = ResultList[0]
    
    # One element which contains the transcript.
    AltList = SpeechRecognitionResult["alternatives"]
    
    # The only element.
    FirstAlt = AltList[0]
    
    # The text content of the audio.
    return FirstAlt["transcript"]

In [ ]:
def Translate(Text, Model) :
    """
     A function which translates Text.
    """
    
    # Create a LanguageTranslatorV3 object using our API key.
    Authenticator = IAMAuthenticator(Keys.IBM_LanguageTranslateAPIKey)
    LT = LanguageTranslatorV3(version = "2018-05-31", authenticator = Authenticator)
    LT.set_service_url(Keys.IBM_LanguageTranslateURL_)
    
    # Perform Translation, get the .json.
    TText = LT.translate(text = Text, model_id = Model).get_result()
    
    # fist list of .json
    TList = TText["translations"]
    
    # first and only translation of the list.
    FirstTranslation = TList[0]
    
    # return the translated string.
    return FirstTranslation["translation"]

In [ ]:
def TextToSpeech(Text, Voice, FileName) :
    """
     A function which converts Text to speech.
    """
    
    # Create a TextToSpeechV1 object to use, using our API key.
    Authenticator = IAMAuthenticator(Keys.IBM_TextToSpeechAPIKey)
    TTS = TextToSpeechV1(authenticator = Authenticator)
    TTS.set_service_url(Keys.IBM_TextToSpeechURL)
    
    # Create a .wav file from the text.
    with open(FileName, "wb") as AudioFile :
        AudioFile.write(TTS.synthesize(Text, accept = "audio/wav", voice = Voice).get_result().content)

In [ ]:
def RecordAudio(FileName) :
    """
     A function which records Audio from our mic and save it to a file.
    """
    # pyaudio recom.
    FRAME_RATE = 44_100      # Frames per second.
    CHUNK = 1024             # Frames read at a time.
    FORMAT = pyaudio.paInt16 # 16bit per frame or a 2 byte int.
    CHANNELS = 1             # 1 samples per frame.
    SECONDS = 5              # Recording time in secs.
    
    # open/close audio streams
    Rec = pyaudio.PyAudio() 
    
    # Configure and open the audio stream.
    AudioStream = Rec.open(format=FORMAT, channels=CHANNELS, 
        rate=FRAME_RATE, input=True, frames_per_buffer=CHUNK)
    
    # Raw bytes of audio stream.
    AudioFrames = [] 
    
    # Message to the user.
    print("Recording 5 Secs of audio.")
    
    # Read 5 seconds of audion in chuck sized pieces.
    for i in range(0, int(FRAME_RATE * SECONDS / CHUNK)) :
        AudioFrames.append(AudioStream.read(CHUNK))
    
    # Message to the user.
    print("Recording stopped.")
    
    # Clean our stream.
    AudioStream.stop_stream()
    AudioStream.close()
    Rec.terminate()
    
    # Save Frames to .wav file.
    with wave.open(FileName, "wb") as OutFile :
        OutFile.setnchannels(CHANNELS)
        OutFile.setsampwidth(Rec.get_sample_size(FORMAT))
        OutFile.setframerate(FRAME_RATE)
        OutFile.writeframes(b''.join(AudioFrames)) # byte string

In [ ]:
def PlayAudio(FileName) :
    """
     A function which plays a .wav
    """
    
    SD = pydub.AudioSegment.from_wav(FileName) 
    pydub.playback.play(SD)

In [ ]:
if __name__ == "__main__" :
    """ Calls our driver program."""
    RunTranslator()